# Cell Number 1 Explanation:
#### apt-get install is a linux command that installs library for your linux system.

#### -q is a flag for apt-get install. It means Quiet. It produces output suitable for logging, omitting progress indicators. More q’s will produce more quiet up to a maximum of two. That’s why it’s written -qq.

#### Sometimes while installing libraries, they may ask for your permission or something else for which you have to give option yes or y. -y is used for that. 

#### software-properties-common => this package contains the common files for software-properties. It manages the repositories that you install the software from (common). This software provides an abstraction of the used apt repositories.

#### module-init-tools is for transitional dummy package. (Often packages in Ubuntu are replaced in newer releases by other packages, or they change name, or are merged into a single package or divided into multiple ones. To ease upgrading, these packages become “dummy” or “transitional” packages, which are practically empty files that depend on the newer packages)

#### add-apt-repository => This commands add the repository to your system.

#### ppa:alessandro-strada/ppa developed by Alessandro Strada => It mounts Google Drive on Ubuntu (via FUSE). FUSE is Filesystem in Userspace which is a simple interface for userspace programs to export a virtual filesystem to the Linux kernel. FUSE also aims to provide a secure method for non privileged users to create and mount their own filesystem implementations.

#### /dev/null is a special file system object that throws away everything written into it. Redirecting a stream into it means hiding an output. 

#### The 2>&1 part means “redirect both the output and the error streams”. Even if your program writes to stderr , that output will not be shown.

#### apt-get update -qq 2>&1 > /dev/null - Updates your system.

#### apt-get -y install -qq google-drive-ocamlfuse fuse - It installs the google-drive-ocamlfuse library. google-drive-ocamlfuse is a FUSE filesystem backed by Google Drive, written in OCaml. It lets you mount your Google Drive on Linux.

from google.colab import auth
auth.authenticate_user()
#### After these, you will be needing some codes for auth. The above piece of code verifies your account to read files which you have access to. Make sure you have permission to read the file!

#### creds = GoogleCredentials.get_application_default() - This returns the Application Default Credentials which are used to identify and authorize the whole application. The following are searched (in order) to find the Application Default Credentials:

google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL =>
#### This is a pipeline command that gets the verified URL after authentication is successfully done and also the the drive is mounted.

#### vcode = getpass.getpass() => Gets a pass from getpass library.

echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} =>
#### It echos or shows the code returned after you clicked the URL returned by the grep URL command

In [ ]:
# Configuration related preprocessing step before mounting the drive

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Cell Number 2 Explanation:
#### Mounting the google drive

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

# Cell Number 3 Explanation:


In [ ]:
"""## Data Pre-processing"""

# Import Necessary Libraries
import os # The OS module in Python provides functions for interacting with the operating system. OS comes under Python's standard utility modules
# Set Directory path for Dataset
os.chdir("/content/drive/My Drive/Face") # changing directory
Dataset='Dataset'
Data_Dir=os.listdir(Dataset) #listdir() returns a list containing the names of the entries in the directory given by path. The list is in arbitrary order. It does not include the special entries '.' and '..' even if they are present in the directory.
print(Data_Dir)


# Cell Number 4 Explanation:



In [ ]:
# Import necessary libraries
import cv2 #OpenCV (Open Source Computer Vision Library) is a library of programming functions mainly aimed at real-time computer vision.
import numpy as np #Numpy adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
#Using the method to_categorical(), a numpy array (or) a vector which has integers that represent different categories, can be converted into a numpy array (or) a matrix which has binary values and has columns equal to the number of categories in the data.
from tensorflow.keras.utils import to_categorical
#Label Binarizer is an SciKit Learn class that accepts Categorical data as input and returns an Numpy array. it encodes the data into dummy variables indicating the presence of a particular label or not. Encoding make column data using Label Binarizer.
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split #Used to split testing and training data

#Dimension of single image i.e. 112x112 pixels
img_rows, img_cols = 112, 112

#Defining 2 empty lists
images = [] #will contain images after grey scaling and b sizing
labels = [] #will contain labels corresponding to the classes of image i.e. with and without mask

#Now we are using nested for loop 
#1st for loop =>  we will get directory path as /content/drive/My Drive/Face/Dataset/With mask
#2nd for loop => to get the list of all image files in the specific directory
for category in Data_Dir:
    folder_path = os.path.join(Dataset, category)
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img) # now join folder_path with each image using seperator /
        img=cv2.imread(img_path) # then reading that image with imread function

        try:
            #Coverting the image into gray scale
            grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            #resizing the gray scaled image into size 56x56 in order to keep size of the images consistent           
            resized_img=cv2.resize(grayscale_img,(img_rows, img_cols))
            images.append(resized_img)
            labels.append(category)
        # Exception Handling in case any error occurs
        except Exception as e:
            print('Exception:',e)
        
images=np.array(images)/255.0 # Normalizing an image
images=np.reshape(images,(images.shape[0],img_rows, img_cols,1)) # resizing without changing data of image array

# Perform one hot encoding on the labels since the label are in textual form so that it can be understood by our deep learning model
lb = LabelBinarizer() # using LabelBinarizer() from sklearn.pre-processing package and creating its object named lb
labels = lb.fit_transform(labels) # fit_transform to apply LabelBinarizer on each label
labels = to_categorical(labels)# to_categorical to convert labels to hot encoded form
labels = np.array(labels) #and converting it into array which is understandable by deep learning model using numpy array function

(train_X, test_X, train_y, test_y) = train_test_split(images, labels, test_size=0.25, 
                                                      random_state=0)# splitting data into testing and training (here test data size 25%)

## A convolutional neural network consists of an input layer, hidden layers and an output layer. In any feed-forward neural network, any middle layers are called hidden because their inputs and outputs are masked by the activation function and final convolution. In a convolutional neural network, the hidden layers include layers that perform convolutions. Typically this includes a layer that does multiplication or other dot product, and its activation function is commonly ReLU. This is followed by other convolution layers such as pooling layers, fully connected layers and normalization layers.

### A Convolutional Neural Network (CNN) architecture has three main parts:

#### A convolutional layer that extracts features from a source image. Convolution helps with blurring, sharpening, edge detection, noise reduction, or other operations that can help the machine to learn specific characteristics of an image.

#### A pooling layer that reduces the image dimensionality without losing important features or patterns.

#### A fully connected layer also known as the dense layer, in which the results of the convolutional layers are fed through one or more neural layers to generate a prediction.

In [ ]:
"""## Build Covolutional Neural Network (CNN) Classification Model"""

# Import Necessary Keras Libraries
#(Here Keras is using tensor flow as backend engine)

# A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.Unlike functional model A Sequential model is not appropriate when: Your model has multiple inputs or multiple outputs. Any of your layers has multiple inputs or multiple outputs.
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D

# Define model paramters
num_classes = 2 #as we have two categories with mask and without mask
batch_size = 32 #batch_size is how many images to pass through a neural network before internal parameters of the model are updated during back propagation
#(Back-propagation i s the essence of neural net training. It is the practice of fine-tuning the weights of a neural net based on the error rate)

# Build CNN model using Sequential API
model=Sequential() # initializng neural network by using Sequential()

#First layer group containing Convolution, Relu and MaxPooling layers
model.add(Conv2D(64,(3,3),input_shape=(img_rows, img_cols, 1))) # here Conv2D means Convolution layer, 64 is filters they are learnable weights learned using back propagation, layers near inputs learns lesser convolution filters while near output they learns more.
# (3,3) is kernal size, it signifies leangth of 1D convolutional window (use to learn filters and reduce volume size)
# input_shape contains input_shape of single image; here 1 represents channel type i.e. grey scale (for colour image it is 3). 

##Activation is done using relu (rectified linear activation function or ReLU for short is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero.)
model.add(Activation('relu'))

# use to reduce the spacial size of representation i.e to reduce amount of parameters and computation in the network progressively. It operates on each feature map independently
model.add(MaxPooling2D(pool_size=(2,2)))

#Second layer group containing Convolution, Relu and MaxPooling layers
#Similarly 2nd layer is created to extract feature of the image. here filter size 128 so max. filters can be extracted from image
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2,2))) 

#Flatten and Dropout Layer to stack the output convolutions above as well as cater overfitting
model.add(Flatten()) #Fully connected layer depicted by Flatten(). it converts convolutional data into 1D array to input into next layer
model.add(Dropout(0.5)) #Dropout use to reduce overfitting in neural network

# Softmax Classifier
model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax')) #softmax activation function use for multi class classification problem, it reports confidence score for each class

print(model.summary()) #Once a model is "built", you can call its summary() method to display its contents:


In [ ]:
# OUTPUT: final output has 2 nodes with mask and without mask

In [ ]:
"""## Plot the Model"""

# Plot the model
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='face_mask_detection_architecture.png') #it takes 2 input arguments 1. model variable name and 2. name with which this image diagram is to me saved


In [ ]:
"""## Train the Model"""

from keras.optimizers import Adam #Here adam is the optimizer. Adams computes individual learning rate for different parameters

epochs = 50 #increase epochs gives better optimized accuracy but training time will increase 

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001), #learning rate = 0.001
              metrics = ['accuracy']) #loss function => compute the quantity that to be reduce during training i.e used to optimize the parameters
# metrics here is used to compute accuracy rate across all predictions

#fitting the model with different paraments and computing the output
fitted_model = model.fit(
    train_X,
    train_y,
    epochs = epochs,
    validation_split=0.25)

## OUTPUT: we got training accuracy 100% and validation accuracy= 84.35

In [ ]:
"""## Plot the Training Loss & Accuracy"""


#history collects the history of model training and keep the details of training and validation data
from matplotlib import pyplot as plt
# Plot Training and Validation Loss
plt.plot(fitted_model.history['loss'],'r',label='training loss')
plt.plot(fitted_model.history['val_loss'],label='validation loss')
plt.xlabel('Number of Epochs') # x axis
plt.ylabel('Loss Value') # y axis
plt.legend()
plt.show()

# Plot Training and Validation Accuracy
plt.plot(fitted_model.history['accuracy'],'r',label='training accuracy')
plt.plot(fitted_model.history['val_accuracy'],label='validation accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy Value')
plt.legend()
plt.show()

## OUTPUT: 

## the validation model is a bit overfitted as it reduces toa certaing value and then increases with increase in epochs

## And accuracy of validation and training model is a bit comparable and is good enough 

In [ ]:
"""## Save or Serialize the Model (so tahat we can reuse it at any time) """

# Save or Serialize the model with the name face_mask_detection_alert_system
model.save('face_mask_detection_alert_system.h5') #h5 is a keras file format use to store deep learning model
